In [228]:
import sqlite3
from sqlite3 import Error
import pandas as pd

In [124]:
conn = sqlite3.connect("./Country_Club_Data/data.db")

In [125]:
cur = conn.cursor()

#### Produce a list of facilities with a total revenue less than 1000. The output of facility name and total revenue, sorted by revenue. 

In [274]:
q = """
        SELECT name as Facility_Name, SUM(cost) AS Revenue FROM 
            (SELECT Facilities.facid, Facilities.name,
            CASE
                WHEN Bookings.memid = CAST(0 as integer)
                    THEN CAST(Facilities.guestcost as integer) * CAST(slots as integer)
                ELSE CAST(Facilities.membercost as integer) * CAST(slots as integer)
            END as 'cost'
            FROM Bookings
            INNER JOIN Facilities ON Bookings.facid = Facilities.facid)
        GROUP BY facid
        HAVING Revenue < 1000
        ORDER BY Revenue DESC
    """
pd.read_sql(q, conn)

,Facility_Name,Revenue
0,Pool Table,270
1,Snooker Table,240
2,Table Tennis,180


#### Produce a report of members and who recommended them in alphabetic surname,firstname order

In [285]:
q = """
    SELECT (surname || ' '|| firstname) as full_name, 
            (SELECT (firstname || ' ' || surname) as recommended_by FROM members AS inner
            WHERE inner.memid = Members.recommendedby) as recommendedby
    FROM Members
    ORDER BY full_name
    """
pd.read_sql(q, conn)

,full_name,recommendedby
0,Bader Florence,Ponder Stibbons
1,Baker Anne,Ponder Stibbons
2,Baker Timothy,Jemima Farrell
3,Boothe Tim,Tim Rownam
4,Butters Gerald,Darren Smith
5,Coplin Joan,Timothy Baker
6,Crumpet Erica,Tracy Smith
7,Dare Nancy,Janice Joplette
8,Farrell David,None
9,Farrell Jemima,None


#### Find the facilities with their usage by member, but not guests

In [286]:
q = """
    SELECT Facilities.name as fac_name, firstname ||' ' ||surname as full_name,COUNT(*) as times_visited from Facilities
    INNER JOIN Bookings on Facilities.facid = Bookings.facid
    INNER JOIN Members on Bookings.memid = Members.memid
    WHERE full_name != 'GUEST GUEST'
    GROUP BY Facilities.name,firstname
    ORDER BY name
    """
pd.read_sql(q, conn)

,fac_name,full_name,times_visited
0,Badminton Court,Anna Mackenzie,30
1,Badminton Court,Anne Baker,10
2,Badminton Court,Burton Tracy,2
3,Badminton Court,Charles Owen,6
4,Badminton Court,Darren Smith,132
...,...,...,...
179,Tennis Court 2,Ponder Stibbons,31
180,Tennis Court 2,Ramnaresh Sarwin,11
181,Tennis Court 2,Tim Rownam,58
182,Tennis Court 2,Timothy Baker,7


#### Find the facilities usage by month, but not guests

In [294]:
q = """
    SELECT strftime('%m', starttime) AS Month,Facilities.name as Facility_name, COUNT(*) AS times_used FROM Bookings
    INNER JOIN Facilities on Facilities.facid = Bookings.facid
    GROUP BY Month, name
    """
pd.read_sql(q, conn)

,Month,Facility_name,times_used
0,07,Badminton Court,56
1,07,Massage Room 1,123
2,07,Massage Room 2,12
3,07,Pool Table,110
4,07,Snooker Table,75
5,07,Squash Court,75
6,07,Table Tennis,51
7,07,Tennis Court 1,88
8,07,Tennis Court 2,68
9,08,Badminton Court,146
